In [1]:
import sys
sys.path.append('../')

from gears import PertData, GEARS

/home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/libpyg.so)
  import torch_geometric.typing
/home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_scatter/_version_cuda.so)
  import torch_geometric.typing
/home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libc

Load data. We use norman as an example.

In [2]:
pert_data = PertData('./data')
pert_data.load(data_name = 'norman')
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...


here1


Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!
Creating dataloaders....
Done!


Create a model object; if you use [wandb](https://wandb.ai), you can easily track model training and evaluation by setting `weight_bias_track` to true, and specify the `proj_name` and `exp_name` that you like.

In [13]:
gears_model = GEARS(pert_data, device = 'cuda:0', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
gears_model.model_initialize(hidden_size = 64)

Downloading...
100%|█████████████████████████████████████| 60.7M/60.7M [00:08<00:00, 7.13MiB/s]
Extracting tar file...
Done!


You can find available tunable parameters in model_initialize via

In [14]:
gears_model.tunable_parameters()

{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

Train your model:

Note: For the sake of demo, we set epoch size to 1. To get full model, set `epochs = 20`.

In [15]:
gears_model.train(epochs = 1, lr = 1e-3)

Start Training...
Epoch 1 Step 1 Train Loss: 0.5317
Epoch 1 Step 51 Train Loss: 0.5139
Epoch 1 Step 101 Train Loss: 0.5127
Epoch 1 Step 151 Train Loss: 0.5347
Epoch 1 Step 201 Train Loss: 0.4490
Epoch 1 Step 251 Train Loss: 0.5341
Epoch 1 Step 301 Train Loss: 0.5533
Epoch 1 Step 351 Train Loss: 0.5931
Epoch 1 Step 401 Train Loss: 0.5317
Epoch 1 Step 451 Train Loss: 0.5329
Epoch 1 Step 501 Train Loss: 0.5112
Epoch 1 Step 551 Train Loss: 0.4384
Epoch 1 Step 601 Train Loss: 0.4467
Epoch 1 Step 651 Train Loss: 0.3978
Epoch 1 Step 701 Train Loss: 0.4199
Epoch 1 Step 751 Train Loss: 0.4449
Epoch 1 Step 801 Train Loss: 0.3939
Epoch 1 Step 851 Train Loss: 0.4299
Epoch 1 Step 901 Train Loss: 0.4656
Epoch 1 Step 951 Train Loss: 0.5928
Epoch 1 Step 1001 Train Loss: 0.5681
Epoch 1 Step 1051 Train Loss: 0.4705
Epoch 1 Step 1101 Train Loss: 0.6066
Epoch 1 Step 1151 Train Loss: 0.4757
Epoch 1 Step 1201 Train Loss: 0.5392
Epoch 1 Step 1251 Train Loss: 0.4281
Epoch 1 Step 1301 Train Loss: 0.4761
Epoch 

Save and load pretrained models:

In [16]:
gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')

Make prediction for new perturbation:

In [17]:
gears_model.predict([['FEV'], ['FEV', 'AHR']])

{'FEV': array([5.3650115e-07, 1.3810029e-02, 4.5100205e-02, ..., 3.4955437e+00,
        3.7515329e-04, 1.3062578e-33], dtype=float32),
 'FEV_AHR': array([ 1.2191668e-06,  3.0627733e-02,  9.5730893e-02, ...,
         2.9410684e+00, -2.5944173e-04,  1.1584434e-33], dtype=float32)}

Gene list can be found here:

In [18]:
gears_model.gene_list[:5]

['RP11-34P13.8', 'RP11-54O7.3', 'SAMD11', 'PERM1', 'HES4']